# Annotation Helper in Action

This notebook walks through an example of the simple tool I built to help annotate documents for a NLP model.

I'm in the process of building a NLP model to extract recipe ingredients from a recipe.  To demonstrate how the helper works I've taken a recipe that needs to be annotated and an existing NER model to walk through the annotation process.

In [1]:
text = "Ingredients 3 tbsp pure maple syrup 3 tbsp reduced sodium soy sauce, or gluten-free soy sauce 1 tbsp sriracha hot sauce 1 clove garlic, smashed 4 wild salmon fillets, skinless (6 oz each) Instructions Combine maple syrup, soy sauce, sriracha and garlic in a small bowl, pour into a gallon sized resealable bag and add the salmon Marinate 20 to 30 minutes, turning once in a while. Preheat air fryer to 400F. Lightly spray the basket with nonstick spray.  Remove the fish from the marinade, reserving and pat dry with paper towels.  Place the fish in the air fryer, in batches, air fry 7 to 8 minutes, or longer depending on thickness of the salmon.  Meanwhile, pour the marinade in a small saucepan and bring to a simmer over medium-low heat and reduce until it thickens into a glaze, 1 to 2 minutes. Spoon over salmon just before eating."


In [2]:
from annotate_helper import AnnotateHelper

annotate = AnnotateHelper(text)

## Suggest Entities

Annotating data can be tiresome, I'll take any chance to simplify the process.  One way to do this is to leverage a simple NER model trained on a handful of documents to suggest entities in the document.  Even though the simple NER model could miss some entities in the document, leveraging the existing model can help streamline the annotation process.

`suggest_entities` will print the text, character span, and entity label for all of the entities identified in the data.  The output from `suggest_entities` will be a list of the detected entities.  Each entity is assigned a unique identifier so entities can be easily referenced.

Running the existing simple model successfully identifies many of the entities in the data, but there are still a few errors that we'll clean up next.

As more documents are annotated you can periodically update the model here to improve the accuracy of the suggestions.

In [3]:
ner_model = "example_spacy_model"

entities = annotate.suggest_entities(ner_model)

54490a905c5b49748d84cc12c4c9eaf8 3 tbsp pure maple syrup INGREDIENT
24ed909c2ff84054887f84e754748d19 3 tbsp reduced sodium soy sauce INGREDIENT
b22b35cc701541d28d39eb072ff0408d -free soy sauce INGREDIENT
054f38966fbe412e9cc78f9d58c7616e 1 tbsp sriracha INGREDIENT
b7307b29a1be43f4b4bfdb4be44b37b1 1 clove garlic INGREDIENT
d7d379d091f346b39faeccba4b7a0130 4 wild salmon fillets INGREDIENT
cfc11259cd9b47a181946ec20a4a4ac2 Instructions Combine maple syrup INGREDIENT
1dee248f105345689aa40d1363db524c Meanwhile, pour INGREDIENT


## Refine Entities (Remove)

The model above seeds the list of labeled entities in the data.  The next step is to remove any incorrect entries from the list of entities.

The `drop_entity` method accepts the unique entity key to be removed.

In this example there are four entities from the suggested list that need to be removed, so let's go ahread and remove them since they are phrases either incorrectly identified as recipe ingredients, or the ingredient is incorrectly split in half.

In [5]:
annotate.drop_entity("24ed909c2ff84054887f84e754748d19") # 3 tbsp reduced sodium soy sauce
annotate.drop_entity("b22b35cc701541d28d39eb072ff0408d") # -free soy sauce
annotate.drop_entity("cfc11259cd9b47a181946ec20a4a4ac2") # Instructions Combine maple syrup
annotate.drop_entity("1dee248f105345689aa40d1363db524c") # Meanwhile, pour

The following entity was removed: 
{'id': '24ed909c2ff84054887f84e754748d19', 'text': '3 tbsp reduced sodium soy sauce', 'start_idx': 36, 'end_idx': 67, 'label': 'INGREDIENT'}
The following entity was removed: 
{'id': 'b22b35cc701541d28d39eb072ff0408d', 'text': '-free soy sauce', 'start_idx': 78, 'end_idx': 93, 'label': 'INGREDIENT'}
The following entity was removed: 
{'id': 'cfc11259cd9b47a181946ec20a4a4ac2', 'text': 'Instructions Combine maple syrup', 'start_idx': 188, 'end_idx': 220, 'label': 'INGREDIENT'}
The following entity was removed: 
{'id': '1dee248f105345689aa40d1363db524c', 'text': 'Meanwhile, pour', 'start_idx': 650, 'end_idx': 665, 'label': 'INGREDIENT'}


## Refine Entities (Add New)

Although the suggested entities step from our simple model is a good place to start, there are going to be some entities in the document that will need to be added.  In this step entity spans are defined for entities that were not detected from the suggested model.

This is a two-step process.  The first step is to get the character locations of a phrase in the text.  You can optionally set the `context_len` parameter to return characters surrounding the entity you are searching for to get an idea for the context the entity appears in the text (by default `context_len` is set to 30).  For each occurrence of the entity there is a prompt asking if the entity should be added.  If you do select to add the entity you must provide a label for the entity.  If the entity you are searching for will only be tagged with one label you can set the `label` parameter to that label to avoid having to manually label each occurrence.

In the previous step the description of the soy sauce was split in half.  Let's go ahead and make sure the whole span gets labeled as an ingredient.

In [6]:
annotate.get_entity_span("3 tbsp reduced sodium soy sauce, or gluten-free soy sauce")

There were 1 matching phrases in the text.

ients 3 tbsp pure maple syrup 3 tbsp reduced sodium soy sauce, or gluten-free soy sauce 1 tbsp sriracha hot sauce 1 c
Do you want to add this to your entity list? (y/n) y
What should the label be for this entity? INGREDIENT
ENTITY ADDED!



## View Annotated Document

Now that the entities have been defined you can view the whole annotated document with all of the entities color coded in the text.  There can be up to 5 different entities marked in the text.  If a phrase is tagged as a part of multiple entities the phrase will be repeated.

If there are multiple entities in the document and you would only like to focus on one of them you can set `focus_entity` to the name of the entity you want to focus on.

To check if all the entities have been identified let's look at what the annotated recipe looks like.  In the document all of the ingredients are highlighted in red to make it easy to review if there are any other entities.

**NOTE:** For one reason or another when this Jupyter Notebook was loaded to GitHub the color formatting was removed.  I'm looking for a solution to this, but the highlighting did work on my local machine.

In [7]:
annotate.view_annotated_document()

COLOR-ENTITY KEY: 
INGREDIENT



Ingredients 3 tbsp pure maple syrup 3 tbsp reduced sodium soy sauce, or gluten-free soy sauce 1 tbsp sriracha hot sauce 1 clove garlic, smashed 4 wild salmon fillets, skinless (6 oz each) Instructions Combine maple syrup, soy sauce, sriracha and garlic in a small bowl, pour into a gallon sized resealable bag and add the salmon Marinate 20 to 30 minutes, turning once in a while. Preheat air fryer to 400F. Lightly spray the basket with nonstick spray.  Remove the fish from the marinade, reserving and pat dry with paper towels.  Place the fish in the air fryer, in batches, air fry 7 to 8 minutes, or longer depending on thickness of the salmon.  Meanwhile, pour the marinade in a small saucepan and bring to a simmer over medium-low heat and reduce until it thickens into a glaze, 1 to 2 minutes. Spoon over salmon just before eating.


## Save the Annotated Document

Once labels have been assigned to each of the entities in the document the annotated document is saved.

Now that we've annotated all the entities in the document let's save the annotations for the document that we can use to improve the NER model.

In [ ]:
annotate.save("annotated/recipe_1.json")